## Set up

In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:

from video_process_utils import *

In [4]:
from keras.models import load_model
import keras.losses
import keras.metrics
keras.losses.loss = keras.losses.mse
keras.metrics.loss = keras.metrics.mse

Using TensorFlow backend.
c:\Users\tanma\miniconda3\envs\pbd\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\tanma\miniconda3\envs\pbd\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\tanma\miniconda3\envs\pbd\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\tanma\miniconda3\envs\pbd\lib\site-packag

In [6]:


from statsmodels.regression.linear_model import OLSResults

In [ ]:
import subprocess

openpose_root_dir = r"D:\KUL\Privacy and big data\openpose\openpose-1.7.0-binaries-win64-gpu-python3.7-flir-3d_recommended\openpose"

command = r"""
& ".\bin\OpenPoseDemo.exe" --video "D:\KUL\Privacy and big data\Project\PBD-Gait\Files\in\input.mp4" --write_json "D:\KUL\Privacy and big data\Project\PBD-Gait\Files\out\keypoints" --display 0 --render_pose 0
"""
result = subprocess.run(["powershell", "-Command", command], capture_output=True, text=True, cwd=openpose_root_dir)

print(result.stdout)
print(result.stderr)



In [7]:
def convert_json2csv(json_dir):
    resL = np.zeros((300,75))
    resL[:] = np.nan
    for frame in range(1,300):
        test_image_json = '%sinput_%s_keypoints.json' %\
            (json_dir, str(frame).zfill(12))

        if not os.path.isfile(test_image_json):
            break
        with open(test_image_json) as data_file:  
            data = json.load(data_file)

        for person in data['people']:
            keypoints = person['pose_keypoints_2d']
            xcoords = [keypoints[i] for i in range(len(keypoints)) if i % 3 == 0]
            counter = 0
            resL[frame-1,:] = keypoints
            break

    #we can save space by dropping rows after the last row that isn't all nan
    check = np.apply_along_axis(lambda x: np.any(~np.isnan(x)),1,resL)
    for i in range(len(check)-1,-1,-1):
        if check[i]:
            break
    return resL[:i+1]

In [9]:
frames = convert_json2csv("../Files/out/keypoints/")
pd.DataFrame(frames)


AttributeError: 'NoneType' object has no attribute 'items'

          0        1         2        3        4         5        6        7   \
0    121.921  91.2449  0.847875  161.962  133.874  0.775835  153.912  132.628   
1    121.886  93.9273  0.631822  161.928  133.936  0.766678  153.899  133.923   
2    121.878  97.8725  0.607743  159.277  133.998  0.775083  149.940  133.984   
3    121.870  99.1929  0.453752  156.634  135.243  0.768227  145.949  135.252   
4    119.249  91.1830  0.666602  157.934  133.931  0.766300  148.621  133.909   
..       ...      ...       ...      ...      ...       ...      ...      ...   
294  208.665  80.5851  0.747435  164.659  123.217  0.862943  175.280  123.208   
295  207.352  85.8944  0.808471  164.628  123.286  0.874698  175.243  123.258   
296  207.366  87.2327  0.819356  164.613  125.954  0.837902  175.234  125.918   
297  205.979  91.2962  0.729357  164.625  129.940  0.848521  175.214  129.923   
298  205.942  92.6027  0.674799  164.632  131.284  0.865866  175.227  132.521   

           8        9   ...

In [10]:
processed_videos = []
processed_video_segments = []

centered_filtered = process_video_and_add_cols(frames)
centered_filtered_noswap = process_video_and_add_cols(frames,
                                swap_orientation=False)

In [11]:
def get_prediction(centered_filtered, col, side = None):
    model = load_model("models/{}_best.pb".format(col))
    correction_model = OLSResults.load("models/{}_correction.pb".format(col))

    maps = {
        "KneeFlex_maxExtension": (-29.4408212510502, 114.8431545843835),
        # "GDI": (32.5990731990764, 80.74813196013362), # doublesided
        "GDI": (36.314492983907, 77.03271217530302), # singlesided
        "gmfcs": (1, 3),
        "speed": (0.0718863507111867, 1.5259117583433834),
        "cadence": (0.222, 1.71556665023985),
        "SEMLS_dev_residual": (-0.8205001909638112, 3.309054961371647)
    }

    def undo_scaling(y,target_min,target_range):
        return y*target_range+target_min

    preds = []

    video_len = centered_filtered.shape[0]
    
    cols = x_columns
    if side == "L":
        cols = x_columns_left
    if side == "R":
        cols = x_columns_right

    samples = []
    for nstart in range(0,video_len-124,31):
        samples.append(centered_filtered[nstart:(nstart+124),cols])
    X = np.stack(samples)
    
    p = model.predict(X)[:,0]
    p = undo_scaling(p, maps[col][0], maps[col][1])
    p = np.transpose(np.vstack([p,np.ones(p.shape[0])]))
    p = correction_model.predict(pd.DataFrame(p))

    return np.mean(p)

In [12]:
def get_all_preds(centered_filtered):
    cols = ["GDI","gmfcs","speed","cadence","SEMLS_dev_residual"]
    return dict([(col, get_prediction(centered_filtered, col)) for col in cols] + [
        ("KneeFlex_maxExtension_L", get_prediction(centered_filtered_noswap, "KneeFlex_maxExtension", "L")),
        ("KneeFlex_maxExtension_R", get_prediction(centered_filtered_noswap, "KneeFlex_maxExtension", "R")),
    ])
    

In [13]:
get_all_preds(centered_filtered)

{'GDI': 84.80461127527384,
 'gmfcs': 1.3795744707902589,
 'speed': 1.0397871673626542,
 'cadence': 1.0731404260071038,
 'SEMLS_dev_residual': -0.4406224531254339,
 'KneeFlex_maxExtension_L': 12.886018471006933,
 'KneeFlex_maxExtension_R': 7.552569266691002}

In [28]:
import gradio as gr
def echo_string(input_string):
    return input_string

# Define your Gradio interface
interface = gr.Interface(
    fn=echo_string, 
    inputs=gr.components.Textbox(label="Enter a string"), 
    outputs=gr.components.Textbox(label="Echoed string")
)

# Launch the interface
interface.launch(debug=True)


Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7872 <> https://04848f583aeecc9f61.gradio.live
Killing tunnel 127.0.0.1:7873 <> https://add99e46d7ac5b47c7.gradio.live
